In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes

In [17]:
X, y = load_diabetes(return_X_y=True)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [20]:
class PCA_custom():

    def __init__(self, n_components):

        self.n_components = n_components
        self.mean = None
        self.variance = None
        self.components = None
    
    def fit(self, X):

        # Center the data
        
        self.mean = X.mean()

        X_centered = X - self.mean

        # Calculate the cov Matrix

        cov_matrix = np.cov(X_centered, rowvar=False)

        # Find the EigenValues and EigeVectors

        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

        # Sort the Eigenvalues and Eigenvectors

        sort_indices = np.argsort(eigenvalues)[::-1]

        eigenvalues = eigenvalues[sort_indices]
        eigenvectors = eigenvectors[: ,sort_indices]

        # Top Components

        self.variance = eigenvalues[self.n_components]

        self.components = eigenvectors[:, :self.n_components]
    
    def transform(self, X):

        X_centered = X - self.mean

        return np.dot(X_centered, self.components)

    def fit_transform(self, X):

        self.fit(X)

        return self.transform(X)


In [21]:
pca_custom = PCA_custom(n_components=5)

X_train = pca_custom.fit_transform(X_train)

X_test = pca_custom.transform(X_test)

In [22]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred_sk = lr.predict(X_test)

In [23]:
from sklearn.metrics import r2_score

r2_score_sk = r2_score(y_test, y_pred_sk)

r2_score_sk

0.4561318079275799

In [28]:
class ElasticNET_custom():

    def __init__(self, learning_rate, epochs, alpha, l1_ratio):

        self.coef_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.alpha = alpha
        self.l1_ratio = l1_ratio

    
    def fit(self, X_train, y_train):

        X_train = np.insert(X_train, 0, 1, axis=1)

        self.coef_ = np.random.randn(X_train.shape[1]) * 0.01

        for i in range(self.epochs):

            coef_slope = np.dot(X_train.T, X_train).dot(self.coef_) - np.dot(y_train.T, X_train) + np.dot(self.alpha, (np.dot(self.l1_ratio, np.sign(self.coef_)) + np.dot((1 - self.l1_ratio), self.coef_)))

            self.coef_ = self.coef_ - (self.lr * coef_slope)

        print(f"Coef_: {self.coef_}")
    
    def predict(self, X_test):

        X_test = np.insert(X_test, 0, 1, axis=1)

        y_pred = np.dot(X_test, self.coef_)
        return y_pred

In [41]:
elastic = ElasticNET_custom(learning_rate=0.001, epochs=2000, alpha=0.01, l1_ratio=0.5)

In [42]:
elastic.fit(X_train, y_train)

y_pred = elastic.predict(X_test)

Coef_: [153.7343522  -21.57682747  11.57705379 -17.8746839   24.86159566
  -1.24001734]


In [43]:
r2_score_c = r2_score(y_test, y_pred)

r2_score_c

0.4561283856111965

In [44]:
print(f"R2 Score SK: {r2_score_sk}")
print(f"R2 Score Custom: {r2_score_c}")

R2 Score SK: 0.4561318079275799
R2 Score Custom: 0.4561283856111965
